In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 182.9 kB/s eta 0:00:0000:0100:46
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 236.4 kB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ba7225591e07717e94aaf9802ea1d4079a10f92fab2666abdc87ebb375bfd5d5
  Stored in directory: /home/jovyan/.cache/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [2]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [ ]:
host='localhost'
port=9999

In [ ]:
# ! nc localhost 9999

In [ ]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step1_1-NetCat") \
    .config("spark.executor.memory", "512mb") \
    .getOrCreate()

In [ ]:
spark.sparkContext.setLogLevel("ERROR")


In [ ]:
lines = spark\
        .readStream\
        .format('socket')\
        .option('host', host)\
        .option('port', port)\
        .load()


In [ ]:
lines.printSchema()

In [ ]:
words_df = lines.select(
        explode(
            split(lines.value, ' ')
        ).alias('word')
    )

In [ ]:
wordCounts = words_df.groupBy('word')\
                      .count()

In [ ]:
query = wordCounts.writeStream\
                      .outputMode('complete')\
                      .format('console')\
                      .start()\
                      .awaitTermination()


In [ ]:
spark.stop()